# BRAIN MRI DETECTION COMPUTER VISION

In [52]:
import os
import numpy as np
import pickle

from skimage.io import imread
from skimage.transform import resize
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [49]:
path = 'D:\Open Classroom\Datasets\Brain MRI Images for Brain Tumor Detection'
categories = ['no', 'yes']

data = []
labels = []

for category_idx, category in enumerate(categories):
    for file in os.listdir(os.path.join(path, category)):
        img_path = os.path.join(path, category, file)
        try:
            img = imread(img_path)
            img = resize(img, (15, 15))
            # Check dimensions after resizing
            if img.shape != (15, 15):
                # print(f"Image {img_path} has unexpected shape after resizing: {img.shape}")
                continue
            flattened_img = img.flatten()
            # Check length of flattened array
            if len(flattened_img) != 15 * 15:
                # print(f"Flattened image {img_path} has unexpected length: {len(flattened_img)}")
                continue
            data.append(flattened_img)
            labels.append(category_idx)
        except Exception as e:
            # print(f"Error processing image {img_path}: {e}")
            pass

        
data = np.asarray(data)
labels = np.asarray(labels)

        
    

In [50]:
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size = 0.2, train_size=0.8, shuffle=True, stratify=labels, random_state=40)

# Build a model classifier
classifier = SVC()

params = [{'gamma':[0.01, 0.001, 0.0001]}, {'C':[1, 10, 100, 1000]}]

grid_search = GridSearchCV(classifier, params)

grid_search.fit(x_train, y_train)


GridSearchCV(estimator=SVC(),
             param_grid=[{'gamma': [0.01, 0.001, 0.0001]},
                         {'C': [1, 10, 100, 1000]}])

In [51]:
# Classifier Model
best_estimator = grid_search.best_estimator_ 

y_predictions = best_estimator.predict(x_test)

acc_score = accuracy_score(y_predictions, y_test)

print('{}% is the accurate score for the model classifier'.format(str(round(acc_score*100))))

100% is the accurate score for the model classifier


In [58]:
pickle.dump(best_estimator, open(os.path.join(path, 'model.p'), 'wb'))

In [65]:
with open(r"D:\Open Classroom\Datasets\Brain MRI Images for Brain Tumor Detection\model.p", 'rb') as f:
    model = pickle.load(f)

def process_image(load_path):
    img = imread(load_path)
    img = resize(img, (15, 15))
    img = img.flatten()
    return img

def classify_img(load_path, model):
    processed_img = process_image(load_path)
    processed_img = processed_img.reshape(1, -1)
    prediction = model.predict(processed_img)
    return prediction


In [82]:
path = r"D:\Downloads\bd937738ad6223a03f8aedcf4920a7_big_gallery.jpeg"
predictions = classify_img(path, model)

print('Predicted class label: ', predictions)

Predicted class label:  [0]
